## Study Dropshipping Discord QA - Capture and Categorize
### Import Dependencies

In [2]:
import os
import boto3
from dotenv import load_dotenv

from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import AIMessage, HumanMessage
from langchain import hub
from langchain.tools import tool
from langchain.agents import initialize_agent
import discord
import asyncio
import nest_asyncio

import json

In [6]:
# Access the environment variables
load_dotenv()
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")
discord_token = os.getenv("DISCORD_TOKEN")

aws_client = boto3.client(
    "bedrock-runtime",
    region_name="us-west-2",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

### Find Possible Servers and Channels to Read From

In [13]:
import discord
import asyncio
import os
import nest_asyncio

nest_asyncio.apply()

def list_guilds_and_channels():
    """
    Lists all guilds and their text channels the bot has access to.
    """
    discord_token = os.getenv("DISCORD_TOKEN")
    if not discord_token:
        print("DISCORD_TOKEN not found in environment.")
        return

    class GuildListerClient(discord.Client):
        async def on_ready(self):
            print(f"\nBot logged in as {self.user}")
            print("Guilds this bot is in:")
            for guild in self.guilds:
                print(f"\n {guild.name} (ID: {guild.id})")
                print("   Channels:")
                for channel in guild.text_channels:
                    print(f"   - #{channel.name} (ID: {channel.id})")
            await self.close()

    intents = discord.Intents.default()
    intents.message_content = True
    client = GuildListerClient(intents=intents)

    async def start_client():
        await client.start(discord_token)

    asyncio.create_task(start_client())


list_guilds_and_channels()


Bot logged in as danDiscScraper#8517
Guilds this bot is in:

 TRUST DROPSHIPPING (ID: 993395850418077867)
   Channels:
   - #ticket-0503 (ID: 1279800462442364998)
   - #ticket-0504 (ID: 1280352999298039940)
   - #ticket-0507 (ID: 1281989855840178271)
   - #ticket-0508 (ID: 1290720290065612881)
   - #ticket-0510 (ID: 1293167285875245127)
   - #ticket-0512 (ID: 1306382424330211444)
   - #ticket-0513 (ID: 1319349502276141148)
   - #ticket-0514 (ID: 1322012379785203812)
   - #ticket-0516 (ID: 1326622414213218417)
   - #ticket-0517 (ID: 1329108726166192169)
   - #ticket-0518 (ID: 1331049394400788491)
   - #ticket-0519 (ID: 1332606130089754706)
   - #rundown-of-discord-in-video-form (ID: 1275543758850494524)
   - #links-to-our-socials (ID: 1275544389904633967)
   - #🔗│our-shopify-link (ID: 1124817675625955379)
   - #📢│announcements (ID: 993571418010824814)
   - #💬│general-chat (ID: 993401342049669150)
   - #🎤│q-and-a (ID: 993401368792547388)
   - #🌱│beginner-chat-and-questions (ID: 10240588

### Use Discord API to Pull Messages from Discord Server

In [80]:
import discord
import asyncio
import nest_asyncio
import os
import time
import json
from typing import List, Dict
from datetime import datetime, timezone

nest_asyncio.apply()

def fetch_discord_threads(channel_name: str, guild_id: int, save_path: str = None, timeout_seconds: int = 600) -> List[Dict]:
    """
    Fetches all messages from a specified Discord channel, preserves reply relationships,
    and groups them into logical threads for question-answer analysis.

    Parameters:
        channel_name: Name of the channel to scrape
        guild_id: Guild (server) ID where the channel is located
        save_path: Optional path to save results as JSON
        timeout_seconds: Max duration before forcing exit

    Returns:
        List of threads, where each thread is a list of message dictionaries
    """
    discord_token = os.getenv("DISCORD_TOKEN")
    if not discord_token:
        raise ValueError("DISCORD_TOKEN not found in environment.")

    class ThreadCollector(discord.Client):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.channel_name = channel_name
            self.guild_id = guild_id
            self.raw_messages = []

        async def on_ready(self):
            print(f"Logged in as {self.user}")
            guild = discord.utils.get(self.guilds, id=self.guild_id)
            if not guild:
                print("Guild not found.")
                await self.close()
                return

            channel = discord.utils.get(guild.text_channels, name=self.channel_name)
            if not channel:
                print(f"Channel '{self.channel_name}' not found.")
                await self.close()
                return

            print(f"Reading from #{channel.name} in '{guild.name}'")
            last_message = None
            start_time = time.time()
            total = 0

            from datetime import datetime, timezone

            cutoff = datetime(2023, 6, 1, tzinfo=timezone.utc)

            while True:
                if time.time() - start_time > timeout_seconds:
                    print("Timeout reached. Ending early.")
                    break

                batch = [msg async for msg in channel.history(limit=100, before=last_message)]
                if not batch:
                    print("No more messages returned. Ending.")
                    break

                # If all messages are before cutoff, stop
                if all(msg.created_at < cutoff for msg in batch):
                    print("Reached messages before June 2023. Stopping early.")
                    break

                for msg in batch:
                    if msg.created_at < cutoff:
                        continue  # Skip older messages

                    self.raw_messages.append({
                        "id": str(msg.id),
                        "author": msg.author.display_name,
                        "content": msg.content,
                        "timestamp": msg.created_at.isoformat(),
                        "reply_to_id": str(msg.reference.message_id) if msg.reference else None,
                        "channel_id": str(msg.channel.id),
                        "attachments": [a.url for a in msg.attachments],
                    })

                last_message = batch[-1]
                total += len(batch)
                if total % 1000 == 0:
                    print(f"Fetched {total} messages so far...")
                await asyncio.sleep(1)


            await self.close()

    intents = discord.Intents.default()
    intents.message_content = True
    client = ThreadCollector(intents=intents)

    loop = asyncio.get_event_loop()
    loop.run_until_complete(client.start(discord_token))

    # Group by threads based on reply_to_id
    message_map = {msg['id']: msg for msg in client.raw_messages}
    thread_groups = []

    seen = set()
    for msg in client.raw_messages:
        if msg["id"] in seen:
            continue

        thread = [msg]
        seen.add(msg["id"])

        current_id = msg["id"]
        # Follow replies
        for reply in client.raw_messages:
            if reply["reply_to_id"] == current_id:
                thread.append(reply)
                seen.add(reply["id"])
                current_id = reply["id"]

        thread_groups.append(thread)

    if save_path:
        with open(save_path, "w") as f:
            json.dump(thread_groups, f, indent=2)

    return thread_groups

In [81]:
#messages = fetch_discord_threads(channel_name="dropship_test2", guild_id=1366864336529789091)
messages = fetch_discord_threads(channel_name="💬│general-chat", guild_id=993395850418077867)

print(f"Fetched {len(messages)} messages.")

Logged in as danDiscScraper#8517
Reading from #💬│general-chat in 'TRUST DROPSHIPPING'
Fetched 1000 messages so far...
Fetched 2000 messages so far...
Fetched 3000 messages so far...
Fetched 4000 messages so far...
Fetched 5000 messages so far...
Fetched 6000 messages so far...
Fetched 7000 messages so far...
Fetched 8000 messages so far...
Fetched 9000 messages so far...
Fetched 10000 messages so far...
Fetched 11000 messages so far...
Fetched 12000 messages so far...
Fetched 13000 messages so far...
Fetched 14000 messages so far...
Fetched 15000 messages so far...
Fetched 16000 messages so far...
Reached messages before June 2023. Stopping early.
Fetched 16700 messages.


In [82]:
from typing import List, Dict

def group_threads_by_reply_tree(thread_groups: List[List[Dict]]) -> List[Dict]:
    """
    Converts raw thread groups into structured threads where each thread has a 'root' and 'replies'.
    Handles nested replies by grouping all related messages together.
    
    Parameters:
        thread_groups: Output of fetch_discord_threads — list of lists of message dicts.
    
    Returns:
        List of threads with 'root' and 'replies'.
    """
    structured_threads = []

    for thread in thread_groups:
        # Create lookup by ID
        id_lookup = {msg['id']: msg for msg in thread}

        # Identify the root message (not replying to anyone in the same thread)
        root_msg = None
        for msg in thread:
            if not msg.get('reply_to_id') or msg['reply_to_id'] not in id_lookup:
                root_msg = msg
                break

        if not root_msg:
            # fallback to first message if no clear root
            root_msg = thread[0]

        # Collect replies (excluding root)
        replies = [msg for msg in thread if msg['id'] != root_msg['id']]

        structured_threads.append({
            'root': f"{root_msg['author']}: {root_msg['content']}",
            'replies': [f"{r['author']}: {r['content']}" for r in replies]
        })

    return structured_threads


In [114]:
structured = group_threads_by_reply_tree(messages)

structured[0:20]

[{'root': 'MikeChamberlin: Yes, 2pm est', 'replies': []},
 {'root': 'ThomasFry: Is the phone call happening today at 2 PM?',
  'replies': ['MikeChamberlin: Yes, 2pm est']},
 {'root': 'Christ Warrior: Nothing really.', 'replies': []},
 {'root': 'MikeChamberlin: Whatve you been working on?',
  'replies': ['Christ Warrior: Nothing really.']},
 {'root': 'Christ Warrior: Pretty good',
  'replies': ['MikeChamberlin: Whatve you been working on?']},
 {'root': 'Dom Dei: Yeah it has been, how’s everything going for you?',
  'replies': ['Christ Warrior: Pretty good']},
 {'root': 'Christ Warrior: Yo guys it’s been a while',
  'replies': ['Dom Dei: Yeah it has been, how’s everything going for you?']},
 {'root': 'MikeChamberlin: I would keep doing research but I will look into iy myself aswell to see if I can find anything on how its performing',
  'replies': []},
 {'root': 'Gio: Should I move on? <@759851216418111529>', 'replies': []},
 {'root': 'Gio: Yeah I found another account that their videos 

In [125]:
import json

output_path = "/Users/dan/calpoly/BusinessAnalytics/GSB570GENAI/studyDropshipping/structured_output.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(structured, f, indent=2, ensure_ascii=False)


### Connect to an LLM (Claude Haiku)

In [88]:
# Select the Bedrock model
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Define generation parameters
model_kwargs =  { 
    "max_tokens": 2048, # maximum tokens to return
    "temperature": 0.0, # creativity
    "top_k": 250,       # restrict to top k tokens
    "top_p": 0.9,       # only sample from set of tokens w/ probability ≤ 0.9
    "stop_sequences": ["\n\nHuman"],
}

# Instantiate the ChatBedrock wrapper
from langchain_aws import ChatBedrock
llm = ChatBedrock(
    client=aws_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# The hub provides sample templates for each agent type
prompt = hub.pull("hwchase17/structured-chat-agent") # this is a template pre-tuned for building a structured chat agent

/Users/dan/calpoly/BusinessAnalytics/GSB570GENAI/gsb570env/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [85]:
len(structured)

16700

In [115]:
prompt_summary = f"""
You are an expert assistant at organizing conversational data into coherent Q&A threads.

Below is a collection of Discord message threads with:
- Root messages (questions/comments/ideas)
- Associated replies
- User identification for each message

Your goal is to transform this raw data into well-structured Q&A pairs that preserve the complete context and information flow. Please:

1. Group messages based on semantic coherence and conversational flow
2. Preserve ALL specific information (URLs, tool names, instructions, code snippets)
3. Consolidate related discussions from the same user, even if they appear as separate threads
4. Maintain the natural question-answer dynamic, including follow-ups and clarifications

Important guidance:
- Consider the full context beyond just punctuation or reply structure
- Messages without question marks can still be questions needing answers
- Pay special attention to who is speaking - MikeChamberlin and DomDei are likely providing authoritative answers
- Messages from other users typically represent questions or requests

When determining thread boundaries:
- Multiple messages from the same user within a brief timeframe likely belong together
- Related follow-ups, clarifications and evolving discussions should be consolidated
- Prioritize conversational coherence over strict reply structure
- Avoid creating separate threads when messages clearly continue a previous discussion

For each coherent thread, please output:

---
Q&A Thread
Question/Topic: [Original question or discussion topic]
Thread: All messages connected to original question in chronological order (This is important so make sure the flow of the conversation is preserved like a human would converse)

---

Only include messages that form part of a coherent conversation. Discard isolated messages without meaningful context or replies.
Please do not include any system messages or instructions in your output.
Remember to maintain the entire original message content, including any URLs or code snippets. You are not summarizing anything, just recording the conversation.

Here are the messages to organize:
{structured[0:250]}
"""

response = llm.invoke(prompt_summary)
print(response.content)

---
Q&A Thread
Question/Topic: Is the phone call happening today at 2 PM?
Thread:
ThomasFry: Is the phone call happening today at 2 PM?
MikeChamberlin: Yes, 2pm est

---
Q&A Thread
Question/Topic: What have you been working on?
Thread:
MikeChamberlin: Whatve you been working on?
Christ Warrior: Nothing really.
Christ Warrior: Pretty good

---
Q&A Thread
Question/Topic: How's everything going?
Thread:
Christ Warrior: Yo guys it's been a while
Dom Dei: Yeah it has been, how's everything going for you?
Christ Warrior: Pretty good

---
Q&A Thread
Question/Topic: Should I move on?
Thread:
Gio: Should I move on? <@759851216418111529>
Gio: Yeah I found another account that their videos go viral but like what you said they focus more on the __doesent have TikTok
MikeChamberlin: I would keep doing research but I will look into iy myself aswell to see if I can find anything on how its performing

---
Q&A Thread
Question/Topic: Thoughts on this product
Thread:
Gio: Thoughts on this
Gio: I feel li

### Form into JSON structure

In [117]:
import re

# Split only on lines that contain exactly three dashes (no more, no less)
thread_blocks = re.split(r"(?m)^\s*---\s*$", response.content)
thread_blocks = [block.strip() for block in thread_blocks if block.strip()]


In [118]:
thread_blocks

['Q&A Thread\nQuestion/Topic: Is the phone call happening today at 2 PM?\nThread:\nThomasFry: Is the phone call happening today at 2 PM?\nMikeChamberlin: Yes, 2pm est',
 'Q&A Thread\nQuestion/Topic: What have you been working on?\nThread:\nMikeChamberlin: Whatve you been working on?\nChrist Warrior: Nothing really.\nChrist Warrior: Pretty good',
 "Q&A Thread\nQuestion/Topic: How's everything going?\nThread:\nChrist Warrior: Yo guys it's been a while\nDom Dei: Yeah it has been, how's everything going for you?\nChrist Warrior: Pretty good",
 'Q&A Thread\nQuestion/Topic: Should I move on?\nThread:\nGio: Should I move on? <@759851216418111529>\nGio: Yeah I found another account that their videos go viral but like what you said they focus more on the __doesent have TikTok\nMikeChamberlin: I would keep doing research but I will look into iy myself aswell to see if I can find anything on how its performing',
 "Q&A Thread\nQuestion/Topic: Thoughts on this product\nThread:\nGio: Thoughts on thi

In [122]:
def build_structuring_prompt(thread_text, thread_id):
    return f"""
You are a dropshipping support assistant.

Below is a Discord Q/A thread. Your job is to:
1. Identify the main question.
2. Preserve the full conversation as `full_thread`, do not summarize the responses ever. Keep all information given.
3. Assign it to a category from this list:
   - Product Research
   - Website Customization
   - Sourcing & Suppliers
   - Shopify Setup / Apps
   - Organic Advertising
   - Paid Advertising
   - Mindset / Motivation
   - General Beginner Questions

Output only a JSON object in this format:

{{
  "id": "qa-thread-{thread_id:03d}",
  "question": "...",
  "full_thread_answer": "...",
  "category": "..."
}}

Here is the thread:
--------------------
{thread_text}
--------------------
"""

In [126]:
structured_threads = []

for i, thread in enumerate(thread_blocks):
    prompt = build_structuring_prompt(thread, i + 1)
    result = llm.invoke(prompt)
    try:
        qa = json.loads(result.content)
        structured_threads.append(qa)
    except json.JSONDecodeError:
        print(f"Error parsing thread {i+1}")

structured_threads

[{'id': 'qa-thread-001',
  'question': 'Is the phone call happening today at 2 PM?',
  'full_thread_answer': 'ThomasFry: Is the phone call happening today at 2 PM?\nMikeChamberlin: Yes, 2pm est',
  'category': 'General Beginner Questions'},
 {'id': 'qa-thread-002',
  'question': 'What have you been working on?',
  'full_thread_answer': 'MikeChamberlin: Whatve you been working on?\nChrist Warrior: Nothing really.\nChrist Warrior: Pretty good',
  'category': 'General Beginner Questions'},
 {'id': 'qa-thread-003',
  'question': "How's everything going?",
  'full_thread_answer': "Christ Warrior: Yo guys it's been a while\nDom Dei: Yeah it has been, how's everything going for you?\nChrist Warrior: Pretty good",
  'category': 'General Beginner Questions'},
 {'id': 'qa-thread-004',
  'question': 'Should I move on?',
  'full_thread_answer': 'Gio: Should I move on? <@759851216418111529>\nGio: Yeah I found another account that their videos go viral but like what you said they focus more on the _

In [ ]:
#messages = fetch_discord_threads(channel_name="dropship_test2", guild_id=1366864336529789091)
messages = fetch_discord_threads(channel_name="💬│general-chat", guild_id=993395850418077867)

print(f"Fetched {len(messages)} messages.")

Logged in as danDiscScraper#8517
Reading from #💬│general-chat in 'TRUST DROPSHIPPING'
Fetched 1000 messages so far...
Fetched 2000 messages so far...
Fetched 3000 messages so far...
Fetched 4000 messages so far...
Fetched 5000 messages so far...
Fetched 6000 messages so far...
Fetched 7000 messages so far...
Fetched 8000 messages so far...
Fetched 9000 messages so far...
Fetched 10000 messages so far...
Fetched 11000 messages so far...
Fetched 12000 messages so far...
Fetched 13000 messages so far...
Fetched 14000 messages so far...
Fetched 15000 messages so far...
Fetched 16000 messages so far...
Reached messages before June 2023. Stopping early.
Fetched 16700 messages.


In [ ]:
# Let's load a staff report for the city of SLO that contains 10 pages of data.  We will chunk the report into 1024 character chunks
# with a 256 character overlap.  We will insert each chunk into the DB with embeddings for similarity search.
with open("data/staff-report.txt", "r") as file:
    text = file.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,  # Adjust the chunk size as needed
    chunk_overlap=256,  # Adjust the overlap between chunks as needed
)

# Split the text into chunks
chunks = text_splitter.split_text(text)

# Now for each chunk open a connection to our DB and insert a record into our generic schema

conn = dbconnection.open_connection_to_db()
try:
    # create a dataframe with new chunk raw text
    for chunk in chunks:
        data_record = {}
        v_embed = generate_vector_embedding(chunk)
        data_record["username"] = MY_USERNAME
        data_record["Embedding"] = chunk
        data_record["Source"] = "Discord"
        data_record["textattribute3"] = ""
        data_record["Category"] = ""
        data_record["textattribute5"] = ""
        data_record["textembedding1"] = v_embed
        insert_record_into_db("rag", data_record, conn)        
        
except Exception as e:
    print(f"An error occurred while inserting records: {e}")
finally:
    if conn:
        conn.close()

In [ ]:
def view_my_data(conn):
    try:
        cur = conn.cursor()

        # SQL statement to delete rows where username is 'bob'
        sql = f"SELECT textattribute1, textattribute2, textattribute3, textattribute4, textattribute5 FROM public.rag WHERE username = '{MY_USERNAME}'"
    
        # Execute the SQL statement
        cur.execute(sql)
        rows = cur.fetchall()

        # grab the cosine scores so we can compute Z score for narrow article selection
        for row in rows:
            print(row[0], row[1], row[2], row[3], row[4])
    
        

        
    except psycopg2.Error as e:
            print("An error occurred:", e)
    finally:
        if conn:
            conn.close()

In [ ]:
conn = dbconnection.open_connection_to_db()
view_my_data(conn)

### Time to Embed and place in Vector DataBase

Store JSON in this form for embedding

In [104]:
from langchain.docstore.document import Document

docs = []

for item in structured_threads: 
    docs.append(
        Document(
    page_content=f"Q: {item['question']}\n{item['full_thread_answer']}",
    metadata={
        "id": item["id"],
        "question": item["question"],
        "category": item["category"]
    }
        )
    )

Use Titan embedding using Bedrock

In [105]:
from langchain_aws.embeddings import BedrockEmbeddings

# This assumes boto3 client for Bedrock is already set up
embedding_model = BedrockEmbeddings(
    client=aws_client,  # your boto3 Bedrock runtime client
    model_id="amazon.titan-embed-text-v1"
)

Store in a vector db with an index file, and a pkl file with content

In [107]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embedding_model)
vectorstore.save_local("gen-chat-test1")


### When we want to use it...

In [108]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.load_local(
    "gen-chat-test1", 
    embeddings=embedding_model,
    allow_dangerous_deserialization=True  # bypass pickle warning)
)

results = vectorstore.similarity_search("I have a useful product, a mechanical dishwashing robot, but I do not know how to market it. What should I do, can you provide an example of a good campaign?", k=3)
for doc in results:
    print(doc.page_content)
    print(doc.metadata)
    print("\n")

Q: Thoughts on selling a ice slushy shaver product
Gio: <@759851216418111529> <@779196497501618196>  I'm planning to sell a ice slushy shaver it's a good product that has gone viral on tik tok and It's my first product I'm trying to sell but I feel like it's dead already since it went viral
MikeChamberlin: Could you link it?
Gio: https://www.tiktok.com/t/ZP86SNxmD/
Gio: Thoughts on this
Gio: I feel like if I sell this product I wouldn't get any sales
MikeChamberlin: This isnt a bad product at all
MikeChamberlin: My only concern is the fact that their only viral videos are less focused on the product and more on the ___ doesnt have tiktok side
MikeChamberlin: Have you ran into any other accounts selling it?
MikeChamberlin: I would keep doing research but I will look into iy myself aswell to see if I can find anything on how its performing
Gio: Yeah I found another account that their videos go viral but like what you said they focus more on the __doesent have TikTok
{'id': 'qa-thread-003

### Feed to Claude

In [110]:
conversation_history = []

In [111]:
user_question = "I have a useful product, a mechanical dishwashing robot, but I do not know hoe to market it. What should I do, can you provide an examplke of a good campaign?"

formatted_history = ''
for turn in conversation_history:
    formatted_history += f"\nUser: {turn['question']}\nAssistant: {turn['answer']}\n"

retrieved_context = "\n\n---\n\n".join([doc.page_content for doc in results])

rag_prompt = f"""
<Role>
You are an expert in helping complete beginners become successful dropshippers. You specialize in: Product Research, Website Overview and Customization, Sourcing and Suppliers, TRUST Dropshipping Group, Mindset, Organic Advertising, Paid Advertising, Shopify Apps
You have learned from an 8-hour course by Mike and Dom, as well as the Study Dropshipping Discord community, TRUST, FAQ threads. You speak clearly, simply, and with enthusiasm, always aiming to help users understand and succeed. You are capable of: Guiding users to their goals,
 explaining concepts without skipping key details, diving deeper if asked, providing examples when relevant. 
</Role>

<Task Flow>
When a user asks a question:
Retrieve relevant information from the internal knowledge base (course material or Discord).

Answer the user’s question step-by-step, as clearly as possible. Include only the response section when you print to the customer:

<information> (facts pulled from course/FAQ) </information>  
<goal> (what the user is trying to accomplish) </goal>  
<difficulty> (how hard or easy this task is) </difficulty> 
<response> (what the user will see based on the steps above, make depth based on difficulty)</response>
If the information is not in the course/FAQ, say:
 "I can’t find any specifics on that question from the course.” Then send the message thread to a new channel called, “unsolved questions.”
</Task Flow>

<Business Specifics>
Your ultimate goal is to help users move forward in their dropshipping journey—ideally to the point where they: Sign up for a Shopify trial using the company’s affiliate link, and then start a paid subscription. By making the process simple, clear, and motivating, you increase the chance that users will become successful dropshippers and long-term subscribers. 
</Business Specifics>

<Examples>
Q: How do I choose a winning product?
A:
 <information>
 Winning products usually share a few characteristics:
They're easy to make eye-catching videos for (think: “TikTok-worthy”).
They spark emotional reactions—especially controversy or surprise.
They solve a problem or make life easier.
 </information>
<goal> To identify a product that will perform well in ads and convert customers. </goal>
 <difficulty> Medium – Requires practice and testing, but tools and examples help. </difficulty>
<response> 

Great question — picking the right product is one of the biggest steps in getting traction with dropshipping.
Start by looking for products that catch attention fast. Ask yourself: Would this stop me from scrolling on TikTok or Instagram? Products that are visually interesting, solve a real problem, or stir some emotion (like surprise or controversy) tend to perform the best in ads.
Next, think about how unique the product feels. If it’s already everywhere, it’ll be tough to stand out. And finally, check the numbers — look for something you can sell for 3x what it costs you to source.
It might take a few tries to land on the right one, but with research tools and inspiration from what’s already working for others, you’ll be able to spot the patterns.
Want help brainstorming or validating a product you’re thinking about?

</response>

</Examples>

<Conversation History>
{formatted_history}
User: {user_question}
Assistant:
</Conversation History>

A user just asked this question:
"{user_question}"

Below are previous Q/A threads from other users. Use them as context to answer the question clearly and step-by-step.

Relevant Threads:
{retrieved_context}

<Reiteration>
You are a friendly, professional dropshipper who wants to grow the community through free, helpful, and clear advice. Be excited to help, break things down step-by-step, and always aim to get the user closer to taking action. 
Only return the <response> without the tags.
</Reiteration>
"""

response = llm.invoke(rag_prompt)
print(response.content)

conversation_history.append({
    "question": user_question,
    "answer": response.content.strip()
})

<response>
That's a really interesting product idea! A mechanical dishwashing robot could definitely fill a need for a lot of people. Here's how I would approach marketing it:

First, focus on highlighting the key benefits and pain points it solves. In your ads and marketing, emphasize how it saves time, reduces effort, and makes kitchen cleanup a breeze. Show it in action so people can visualize how it would fit into their daily lives.

Next, think about the target audience. Who would benefit most from this product? Busy families, apartment dwellers, elderly folks - get specific on who you're trying to reach. That will help you craft messaging and choose the right advertising channels.

For the campaign itself, I'd recommend a mix of organic and paid strategies. On the organic side, create shareable video content that demonstrates the robot in use. Post it on TikTok, Instagram Reels, YouTube Shorts - anywhere your target customers are spending time. Leverage influencers and user-gener

In [112]:
retrieved_context

'Q: Thoughts on selling a ice slushy shaver product\nGio: <@759851216418111529> <@779196497501618196>  I\'m planning to sell a ice slushy shaver it\'s a good product that has gone viral on tik tok and It\'s my first product I\'m trying to sell but I feel like it\'s dead already since it went viral\nMikeChamberlin: Could you link it?\nGio: https://www.tiktok.com/t/ZP86SNxmD/\nGio: Thoughts on this\nGio: I feel like if I sell this product I wouldn\'t get any sales\nMikeChamberlin: This isnt a bad product at all\nMikeChamberlin: My only concern is the fact that their only viral videos are less focused on the product and more on the ___ doesnt have tiktok side\nMikeChamberlin: Have you ran into any other accounts selling it?\nMikeChamberlin: I would keep doing research but I will look into iy myself aswell to see if I can find anything on how its performing\nGio: Yeah I found another account that their videos go viral but like what you said they focus more on the __doesent have TikTok\n\n-